In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os


In [2]:
from google.colab import files
uploaded = files.upload()

Saving happy-or-sad.zip to happy-or-sad.zip


In [3]:
import zipfile
local_zip = '/content/happy-or-sad.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content/happy-or-sad')

In [4]:
zip_ref.close()

import os

# Directory with training happy pictures
train_happy_dir = os.path.join('./happy-or-sad/happy')

# Directory with training human pictures
train_sad_dir = os.path.join('./happy-or-sad/sad')

In [5]:
train_happy_names = os.listdir(train_happy_dir)
print(f'TRAIN SET HAPPY: {train_happy_names[:10]}')

train_sad_names = os.listdir(train_sad_dir)
print(f'TRAIN SET SAD: {train_sad_names[:10]}')

TRAIN SET HAPPY: ['happy1-06.png', 'happy1-05.png', 'happy1-03.png', 'happy1-13.png', 'happy1-15.png', 'happy1-00.png', 'happy1-16.png', 'happy1-19.png', 'happy2-19.png', 'happy2-05.png']
TRAIN SET SAD: ['sad1-03.png', 'sad1-09.png', 'sad1-06.png', 'sad1-19.png', 'sad2-12.png', 'sad2-16.png', 'sad2-13.png', 'sad1-16.png', 'sad1-04.png', 'sad1-18.png']


In [6]:
print(f'Total happy images: {len(os.listdir(train_happy_dir))}')
print(f'Total sad images: {len(os.listdir(train_sad_dir))}')

Total happy images: 40
Total sad images: 40


In [15]:
model = tf.keras.models.Sequential([
    #first convolution
    tf.keras.layers.Conv2D(16, (3, 3), padding = 'valid', input_shape = (150, 150, 3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    #second convolution
    tf.keras.layers.Conv2D(32, (3, 3), padding = 'valid', activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    #third convolution
    tf.keras.layers.Conv2D(64, (3, 3), padding = 'valid', activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    #fully connected layers
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid'),
])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 148, 148, 16)      448       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 74, 74, 16)       0         
 2D)                                                             
                                                                 
 conv2d_4 (Conv2D)           (None, 72, 72, 32)        4640      
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 36, 36, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 34, 34, 64)        18496     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 17, 17, 64)      

In [16]:
#compile the model
from tensorflow.keras.optimizers import RMSprop
model.compile(loss = 'binary_crossentropy', optimizer = RMSprop(learning_rate = 0.001), metrics = ['accuracy'])

In [17]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_directory(
        "/content/happy-or-sad",  
        target_size=(150, 150), 
        batch_size=5,
        class_mode='binary')

Found 80 images belonging to 2 classes.


In [18]:
#construct callback
# import tf.keras.callbacks.Callback as cb
DESIRED_ACCURACY = 0.999
class my_callback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs = {}):
    if(logs.get('accuracy') >= DESIRED_ACCURACY):
      print("Accuracy reached 99.9%. stopping the training now")
      self.model.stop_training = True

In [19]:
#fit the model
callbacks = my_callback()
history = model.fit_generator(
      train_generator,
      steps_per_epoch=16,  
      epochs=15,
      verbose=1,
      callbacks=[callbacks])
    

Epoch 1/15


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


16/16 [==============================] - 4s 173ms/step - loss: 0.9443 - accuracy: 0.6000
Epoch 2/15
16/16 [==============================] - 3s 173ms/step - loss: 0.3187 - accuracy: 0.8375
Epoch 3/15
16/16 [==============================] - 3s 176ms/step - loss: 0.1946 - accuracy: 0.9125
Epoch 4/15
16/16 [==============================] - 3s 172ms/step - loss: 0.0721 - accuracy: 0.9875
Epoch 5/15
16/16 [==============================] - 3s 174ms/step - loss: 0.0869 - accuracy: 0.9375
Epoch 6/15
16/16 [==============================] - 3s 173ms/step - loss: 0.0388 - accuracy: 0.9875
Epoch 7/15
16/16 [==============================] - 3s 174ms/step - loss: 0.0036 - accuracy: 1.0000


In [30]:
#making predictions on our own image


import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = '/content/' + fn
  img = image.load_img(path, target_size=(150, 150))
  x = image.img_to_array(img)
  x /= 255
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(classes[0])
  if classes[0]>0.5:
    print(fn + " is heppi")
  else:
    print(fn + " is a sed")

Saving sad1-16.png to sad1-16.png


IndexError: ignored